In [1]:
import pandas as pd 
import re

fight_df = pd.read_csv('ufc_fight_data copy.csv')
event_df = pd.read_csv('ufc_event_data.csv')
finishes_df = pd.read_excel('Finishes by Year-1.xlsx')

In [2]:
# Merge fight and event datasets on 'event_id' to analyze fights over time
merged_event_id = pd.merge(fight_df, event_df, on="event_id", how="left")

# Select only relevant columns
relevant_columns = ['event_id', 'result_details', 'gender', 'weight_class', 'event_date']
filtered_data = merged_event_id[relevant_columns].copy()

# Clean 'result_details' by removing newline characters
filtered_data['result_details'] = filtered_data['result_details'].str.replace('\n', ' ' ,regex=True)

# Convert 'event_date' to datetime format and extract only the year
filtered_data['event_date'] = pd.to_datetime(filtered_data['event_date'])
filtered_data['year'] = filtered_data['event_date'].dt.year

# Group data by year, result details, and weight class, then count occurrences
finishes_by_year = filtered_data.groupby(['year', 'result_details', 'weight_class']).size().reset_index(name='count')
finishes_by_year    

,year,result_details,weight_class,count
0,1994,Arm Triangle From Mount,Open Weight,1
1,1994,Armbar,Open Weight,1
2,1994,Armbar From Bottom Guard,Open Weight,2
3,1994,Armbar From Mount,Open Weight,2
4,1994,Armbar On Ground,Open Weight,1
...,...,...,...,...
2869,2023,Unanimous,Women's Strawweight,11
2870,2023,to Arm Injury,Featherweight,1
2871,2023,to Injury,Middleweight,1
2872,2023,to Leg Injury,Lightweight,1


In [3]:
def categorize_result(detail):
    # Ensure the detail is lowercase for case-insensitive comparison
    if not isinstance(detail, str):
        return "Uncategorized"
  # Standardize formatting: remove extra spaces and convert to lowercase
    detail = re.sub(r'\s+', ' ', detail.strip().lower())  

   
# Categorization rules
    if "unanimous" in detail:
        return "Decision - Unanimous"
    elif "split" in detail:
        return "Decision - Split"
    elif "majority" in detail:
        return "Decision - Majority"
    elif "headbutt" in detail:
        return "Headbutts"
    elif "stomps" in detail:
        return "Stomps"
    elif "fatigue" in detail:
        return "Fatigue"
    elif "corner stoppage" in detail:
        return "Corner Stoppage"
    elif "other" in detail:
        return "Other"
    elif "gi choke" in detail:
        return "Gi Choke"
    elif "schultz" in detail:
        return "Schultz"
    elif "punches" in detail and "head" in detail:
        return "Punches to Head"
    elif "rear naked choke" in detail:
        return "Rear Naked Choke"
    elif "guillotine" in detail:
        return "Guillotine"
    elif (
        ("punch" in detail and "ground" in detail) or
        ("punch" in detail and "guard" in detail) or
        ("elbow" in detail and "ground" in detail) or
        ("elbow" in detail and "guard" in detail) or
        ("punch" in detail and "back" in detail) or
        ("punch" in detail and "mount" in detail)
    ):
        return "Ground and Pound"
    elif "head" in detail and "kick" in detail:
        return "Head Kick"
    elif "body" in detail and "punch" in detail:
        return "Body Punch"
    elif "arm triangle" in detail:
        return "Arm Triangle"
    elif "elbow" in detail:
        return "Elbow"
    elif "armbar" in detail:
        return "Armbar"
    elif "knee" in detail and "head" in detail:
        return "Knee to Head"
    elif "triangle choke" in detail:
        return "Triangle Choke"
    elif "injury" in detail:
        return "Injury"
    elif "flying knee" in detail:
        return "Flying Knee"
    elif "anaconda choke" in detail:
        return "Anaconda Choke"
    elif "d'arce choke" in detail:
        return "D'Arce Choke"
    elif "body" in detail and "kick" in detail:
        return "Body Kick"
    elif "triangle armbar" in detail:
        return "Triangle Armbar"
    elif "heel hook" in detail:
        return "Heel Hook"
    elif "twister" in detail:
        return "Twister"
    elif "kimura" in detail:
        return "Kimura"
    elif "neck crank" in detail:
        return "Neck Crank"
    elif "spinning back fist" in detail:
        return "Spinning Back Fist"
    elif "cut" in detail:
        return "Facial Cut"
    elif "illegal strike" in detail:
        return "Illegal Strike"
    elif "suloev stretch" in detail:
        return "Suloev Stretch"
    elif "calf slicer" in detail:
        return "Calf Slicer"
    elif "slam" in detail:
        return "Slam"
    elif "north south choke" in detail:
        return "North South Choke"
    elif "knee" in detail and "body" in detail:
        return "Knee to Body"
    elif "spinning back kick" in detail:
        return "Spinning Back Kick"
    elif "kneebar" in detail:
        return "Kneebar"
    elif "inverted triangle" in detail:
        return "Inverted Triangle"
    elif "ankle lock" in detail:
        return "Ankle Lock"
    elif "von flue choke" in detail:
        return "Von Flue Choke"
    elif "keylock" in detail:
        return "Keylock"
    elif "scarf hold" in detail:
        return "Scarf Hold"
    elif "bulldog choke" in detail:
        return "Bulldog Choke"
    elif "forearm choke" in detail:
        return "Forearm Choke"
    elif "ezekiel choke" in detail:
        return "Ezekiel Choke"
    elif "omoplata" in detail:
        return "Omoplata"
    elif "shoulder choke" in detail:
        return "Shoulder Choke"
    elif "peruvian necktie" in detail:
        return "Peruvian Necktie"
    elif "toe hold" in detail:
        return "Toe Hold"
    else:
        return "Uncategorized"


# Clean the result_details column
finishes_by_year['result_details'] = finishes_by_year['result_details'].str.strip().str.replace(r'\s+', ' ', regex=True).str.lower()
# Apply categorization to result details
finishes_by_year['category'] = finishes_by_year['result_details'].apply(categorize_result)

# Group by category, weight_class, and year while summing counts
grouped_data = finishes_by_year.groupby(['category', 'weight_class', 'year'], as_index=False)['count'].sum()

# Display the grouped data
print(grouped_data)

# Check remaining Uncategorized rows
uncategorized_rows = finishes_by_year[finishes_by_year['category'] == "Uncategorized"]
print(uncategorized_rows[['result_details', 'weight_class', 'year']])
 

            category         weight_class  year  count
0     Anaconda Choke         Bantamweight  2017      2
1     Anaconda Choke         Bantamweight  2022      1
2     Anaconda Choke        Featherweight  2012      1
3     Anaconda Choke        Featherweight  2014      1
4     Anaconda Choke        Featherweight  2017      1
...              ...                  ...   ...    ...
2069  Von Flue Choke    Light Heavyweight  2019      1
2070  Von Flue Choke    Light Heavyweight  2021      1
2071  Von Flue Choke          Lightweight  2017      1
2072  Von Flue Choke         Welterweight  2006      1
2073  Von Flue Choke  Women's Strawweight  2022      1

[2074 rows x 4 columns]
                 result_details         weight_class  year
20                           to          Open Weight  1994
49     to submission to strikes          Open Weight  1995
65    punch to head at distance          Open Weight  1996
66      punch to head in clinch          Open Weight  1996
79                  